<a href="https://colab.research.google.com/github/EduardMozharov/ML_Neural_networks/blob/main/Project_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project "Emotion Detection"
Итоговая работа курса SKILLBOX:  
Data Science. ML. Средний уровень (Нейронны сети))

# Задание
**Цель** - реализовать нейронную сеть, распознающую эмоции

**Задачи:**
1. Работа требует реализации алгоритма, с помощью которого будет происходить заданная классификация. Приветствуется использование архитектур свёрточных нейронных сетей, разобранных во время теоретических занятий. Время инференса сети на Google Colab не должно превышать 0,33 секунды (3 кадра в секунду).

2. Создать скрипт, который будет работать с их веб-камерами и выводить на экран текущую эмоцию.


**Дополнительно:**  
1. Используйте подходы по искусственному увеличению датасета для обучения, а именно — аргументации, рассмотренные в курсе.
2. Используйте продвинутые техники обучения, такие как finetuning и другие, рассмотренные в курсе.
3. Пишите структурный код (основная логика кода должна быть вынесена в классы/методы/функции + осмысленный нейминг сущностей и комментарии).
4. Проведите эксперименты с valence-arousal разложением эмоций, когда модель обучается не на самих эмоциях, а на их разложении по этим двум компонентам. Информация о подобном разложении есть в дополнительных материалах.

 
Работающий прототип из пункта «Улучшение и дополнения к заданию» прибавляет один балл к итоговой оценке и добавляет в портфолио ценный ML-проект.




# 1 Загрузим необходимые библиотеки

In [ ]:
DO_IT = True # Выполнять ли особые части кода

In [ ]:
!pip install livelossplot

# Для работы оптимизатора TensorRT может потребоваться предыдущая версия tensorflow
#!pip install -U tensorflow==2.7.0
#!pip install keras==2.6.*

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
print('TensorFlow version:', tf.__version__)
from livelossplot.tf_keras  import PlotLossesCallback

import timeit

TensorFlow version: 2.9.2


#2. Загрузим данные и рассмотрим их

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Один раз загрузим и разархивируем данные на личный диск,
# чтобы каждый раз заново не загружать данные
path = '/content/drive/MyDrive/SKILLBOX/FinalWorks/ML/'
os.chdir(path)
!unzip -q train.zip
!unzip -q test_kaggle.zip

**image_path** - строка, являющаюся путем до изображения в случае обучения или названием изображения в случае теста  
**emotion** - строка, характеризующая эмоцию  

  
В данном задании требуется предсказать 9 базовых эмоций, таких как:  
* **neutral** - нейтральная эмоция
* **anger**- гнев, злость
* **contempt** - презрение
* **disgust** - отвращение
* **fear** - страх
* **happy** - веселый
* **sad** - грусть
* **surprise** - удивленность
* **uncertain** - неуверенность

#3. Модель

В ходе выполнения работы придерживаемся следующих правил по оптимизации времения выполнения сети:
1) Оптимизация модели при обучении:
   * 1.1)  От простой модели к сложной.
   * 1.2) Не заставлять учить то, что задается явно.
   * 1.3) Понизить ширину сети за счет Conv(1х1).
   * 1.4) Заменить Conv+Pool2D на Strided Conv.
   * 1.5) Использовать активацию Relu по возможности.
   * 1.6) Количество карт признаков делать кратным 8.
   * 1.7) Применить поканальную сепарабельную свертку (Depth-wise separable Conv).
   * 1.8) Применить Прореживание (Puring).
   * 1.9) Другое... 

2) Оптимизация модели после обучения: с помощью TensorRT

#3.1 Проверим время работы одного прохода нейронной сети

In [ ]:
%%timeit -n 10 -r 10
q_opt = model_opt_3(np.random.rand(1, inp_size, inp_size, 3).astype(np.float32))

#4. Обучение модели

#5. Оптимизация модели